In [ ]:
# !pip install datasets
# !pip install evaluate
# !pip install bert_score
# !pip install bitsandbytes
# !pip install peft
# !pip install textstat
# # !pip install -U bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import re, time, random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import torch
import torch.nn.functional as F
import transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    T5ForConditionalGeneration,
    T5Tokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
from peft import (
    PeftModel,
    PeftConfig,
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    TaskType
)
import evaluate
from bert_score import score as bert_score
import textstat
from datasets import load_dataset, Dataset
from tqdm import tqdm
import bitsandbytes

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Load in LLaMA
def load_llama_model():
    model_name = "meta-llama/Llama-3.2-3B-Instruct"
    model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return model, tokenizer

#Preprocess the Reveal Dataset
def preprocess_datasets():
    openbookqa = load_dataset("openbookqa", "main")
    reveal = load_dataset("google/reveal")

    def filter_bad_chains(example):
        if 'explanation' in example and example['explanation']:
            return len(example['explanation'].split()) > 5
        return False

    filtered_openbookqa = openbookqa.filter(filter_bad_chains)
    return filtered_openbookqa, reveal




In [ ]:
# !huggingface-cli login


In [ ]:
llama_model, llama_tokenizer = load_llama_model()


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
#Load Datasets
dataset_openbookqa, dataset_reveal = preprocess_datasets()


README.md:   0%|          | 0.00/9.06k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/496k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/58.2k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4957 [00:00<?, ? examples/s]

...

Generating eval split:   0%|          | 0/4956 [00:00<?, ? examples/s]

Generating open split:   0%|          | 0/1146 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4957 [00:00<?, ? examples/s]

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
#Verifier Finetuned in "Finetune_Gemma2b.ipynb" notebook, loading it in here
def load_finetuned_verifier():

    peft_model_path = "/content/drive/MyDrive/Colab Notebooks/DS266/Final_Project/gemma_verifier_enhanced_lora_final_100"

    # Load PEFT configuration + Base Verifier Model (FP16)
    peft_config = PeftConfig.from_pretrained(peft_model_path)
    base_model = AutoModelForCausalLM.from_pretrained(
        peft_config.base_model_name_or_path,
        torch_dtype=torch.float16,
        device_map="auto"
    )

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(peft_model_path)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        base_model.config.pad_token_id = tokenizer.pad_token_id

    # Load fine-tuned LoRA layers onto the base model
    model = PeftModel.from_pretrained(base_model, peft_model_path)

    # Prediction Function with Few shot examples
    def predict_validity(question, step, previous_steps=None, step_type="Unknown"):
        is_logical_step = "Logical" in step_type

    # Prediction Function with Few shot examples
        previous_steps_text = ""
        if previous_steps and len(previous_steps) > 0:
            previous_steps_text = "Previous steps:\n" + "\n".join(f"- {s}" for s in previous_steps if s) + "\n\n"

        # Prompt Engineering for Verifier (including Few-Shot Examples)
        if is_logical_step:
            instruction = (
                "You are a reasoning verifier. Your task is to determine if a reasoning step is logically correct "
                "given the context of a question and previous reasoning steps. Focus on whether the step follows "
                "logically from what came before, not just whether it's factually accurate on its own."
            )
            few_shot_example = (
                "Example 1:\n"
                "Question: What is 5 + 7?\n"
                "Previous steps:\n"
                "- 5 + 7 can be calculated directly.\n"
                "Reasoning Step: The sum of 5 and 7 is 12.\n"
                "Is the reasoning step valid? Yes, this reasoning step is valid.\n\n"
                "Example 2:\n"
                "Question: How many planets are in our solar system?\n"
                "Previous steps:\n"
                "- There are 8 recognized planets in our solar system.\n"
                "- Pluto used to be considered the 9th planet.\n"
                "Reasoning Step: Therefore, there are 9 planets in our solar system.\n"
                "Is the reasoning step valid? No, this reasoning step is invalid. The previous step states that Pluto is no longer considered a planet.\n\n"
            )
        else:
            instruction = (
                "You are a reasoning verifier. Your task is to determine if a reasoning step is valid "
                "given the context of a question. Evaluate whether the step is reasonable and contributes "
                "to answering the question."
            )
            few_shot_example = (
                "Example 1:\n"
                "Question: What is the capital of France?\n"
                "Reasoning Step: Paris is the capital of France.\n"
                "Is the reasoning step valid? Yes, this reasoning step is valid.\n\n"
                "Example 2:\n"
                "Question: What is 2+2?\n"
                "Reasoning Step: 2+2=5\n"
                "Is the reasoning step valid? No, this reasoning step is invalid.\n\n"
            )

        # Format task
        task = (
            f"Question: {question}\n"
            f"{previous_steps_text}"
            f"Reasoning Step: {step}\n"
            f"Is the reasoning step valid?"
        )

        # Combine everything into full prompt
        input_text = f"{instruction}\n\n{few_shot_example}{task}"

        inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=150,
                temperature=0.1,
                do_sample=False,
                top_p=0.95,
            )

        # Extract the prediction
        prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extract just the prediction part
        response = prediction.split("Is the reasoning step valid?")[-1].strip()
        contains_explanation = len(response.split()) > 5

        #Case 1:
        # Determine if valid or invalid. This is a manual approach of verification indication.
        valid_indicators = ["yes", "valid", "correct", "accurate", "true"]
        invalid_indicators = ["no", "invalid", "incorrect", "wrong", "false", "error", "mistake"]
        has_valid = any(indicator in response.lower() for indicator in valid_indicators)
        has_invalid = any(indicator in response.lower() for indicator in invalid_indicators)

        #Case 2:
        # Look for specifc phrases, again manual approach to verification indication.
        if (has_valid and has_invalid) or (not has_valid and not has_invalid):
            # Look for specific phrases
            if "yes, this reasoning step is valid" in response.lower():
                return True
            elif "no, this reasoning step is invalid" in response.lower():
                return False

            #Case 3:
            # If there's an explanation, do more nuanced analysis
            if contains_explanation:

                # Count positive and negative indicators to determine final output
                valid_count = sum(response.lower().count(indicator) for indicator in valid_indicators)
                invalid_count = sum(response.lower().count(indicator) for indicator in invalid_indicators)

                if valid_count > invalid_count:
                    return True
                elif invalid_count > valid_count:
                    return False

            #Case 4:
            # if "yes" or "no" is at the start
            if response.lower().startswith("yes"):
                return True
            elif response.lower().startswith("no"):
                return False

        #Case 5:
        # If only valid indicators
        if has_valid and not has_invalid:
            return True

        #Case 6:
        # If only invalid indicators
        if has_invalid and not has_valid:
            return False

        #Case 7:
        # Stay conservative and return this as False (better to have a False Negative than False Positive)
        return False

    return model, tokenizer, predict_validity


In [ ]:


verifier_model, verifier_tokenizer, predict_validity = load_finetuned_verifier()


['README (1).md', 'adapter_model.safetensors', 'config.json', 'special_tokens_map.json', 'tokenizer.model', 'tokenizer.json', 'adapter_config.json']


config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
# Extract individual reasoning steps from the chain with improved filtering
def extract_reasoning_steps(reasoning_chain):

    # Skip question prefix if present
    if "Q:" in reasoning_chain and "Let's reason step by step" in reasoning_chain:
        content = reasoning_chain.split("Let's reason step by step")[-1].strip()
    else:
        content = reasoning_chain

    # Check for step numbering (Step 1, Step 2) or numbered lists (1., 2.)
    if "Step" in content or any(re.search(r"\b\d+[\.\:]", line) for line in content.split("\n")):
        lines = content.split("\n")
        steps, current_step, in_step = [], "", False

        for line in lines:
            line = line.strip()
            if re.search(r"\bStep\s+\d+[\.\:]", line) or re.search(r"^\d+[\.\:]", line):
                if current_step and in_step:
                    steps.append(current_step.strip())
                current_step, in_step = line, True
            elif in_step and line:
                current_step += " " + line

        if current_step and in_step:
            steps.append(current_step.strip())

        if steps:
            return steps

    # Check for markdown headers (## Sectioned Steps)
    if "##" in content:
        sections = content.split("##")
        steps = [section.strip() for section in sections[1:] if section.strip()]
        if steps:
            return steps

    # Default to sentence splitting
    sentences = re.split(r'(?<=[.!?])\s+', content)
    return [s.strip() for s in sentences if s.strip() and len(s) > 10]


#Compute the average pairwise cosine similarity between reasoning steps.
def coherence_score(steps):
    if len(steps) < 2:
        return 1.0  # Fully coherent if single step

    vectorizer = TfidfVectorizer().fit_transform(steps)
    similarity_matrix = cosine_similarity(vectorizer, vectorizer)
    return similarity_matrix.mean()  # Average pairwise similarity


#Runs iterative refinement up to 3 times, fixing invalid steps and tracking performance.
def iterative_refinement(generator_model, generator_tokenizer, verifier_model, verifier_tokenizer,
                         question, predict_validity):
    input_prompt = f"Question: {question}\n Reason step by step."
    best_reasoning, refinement_attempts = None, 0
    start_time = time.time()  # Start tracking refinement time

    for iteration in range(3):  # Max 3 refinements
        refinement_attempts += 1

        # Generate reasoning
        inputs = generator_tokenizer(input_prompt, return_tensors="pt").to("cuda")
        outputs = generator_model.generate(**inputs, max_length=200)
        reasoning_chain = generator_tokenizer.decode(outputs[0], skip_special_tokens=True)
        print(f"Iteration {iteration + 1}: Generated Reasoning:\n{reasoning_chain}\n{'-'*50}")


        # Extract individual steps
        steps = extract_reasoning_steps(reasoning_chain)

        # Verify each step
        invalid_steps = []
        print(f"Extracted Steps for Iteration {iteration + 1}:")
        for i, step in enumerate(steps):
            print(f"Step {i+1}: {step}")
            previous_steps = steps[:i] if i > 0 else None
            step_type = "Attribution step." if i == 0 else "Logical step."
            is_valid = predict_validity(question, step, previous_steps, step_type)

            if not is_valid:
                invalid_steps.append((i, step))
        print('-' * 50)

        # If all steps are valid, return reasoning
        if not invalid_steps:
            refinement_time = time.time() - start_time
            return reasoning_chain, refinement_attempts, refinement_time

        # Store best reasoning attempt (if less than half steps are invalid)
        if best_reasoning is None and len(invalid_steps) <= len(steps) / 2:
            best_reasoning = reasoning_chain

        # Initialize feedback before appending to it
        feedback = ""
        for idx, step in invalid_steps:
            feedback += f"- Step {idx+1} is invalid: \"{step}\"\n"
        feedback += (
    "For each invalid step, rewrite it to ensure it follows from the previous steps. Identify incorrect assumptions and provide a revised version. "
    "Break down each step logically, verify assumptions, and ensure consistency in your thought process. "
    "If any gaps, ambiguities, or contradictions exist, correct them by providing clearer justifications. ")

        input_prompt = f"Q: {question}\n{feedback}\n"



    refinement_time = time.time() - start_time
    return best_reasoning if best_reasoning else reasoning_chain, refinement_attempts, refinement_time







def run_experiment(generator_model, generator_tokenizer, verifier_model, verifier_tokenizer,
                   dataset, num_samples=100, predict_validity=None):

    results = []

    # Convert HF Dataset to list of dictionaries
    if isinstance(dataset, Dataset):
        dataset = dataset.to_pandas().to_dict(orient="records")  # Convert Dataset -> Pandas -> List of Dicts
    if isinstance(dataset, dict):
        dataset = list(dataset.values())
    elif isinstance(dataset, pd.DataFrame):
        dataset = dataset.to_dict(orient="records")

    dataset_sample = random.sample(dataset, min(num_samples, len(dataset))) if isinstance(dataset, list) else dataset
    for example in tqdm(dataset_sample):
        question = example.get("question", "")
        if not question:
            continue

        try:
            # Generate initial reasoning
            inputs = generator_tokenizer(f"Q: {question}\nLet's reason step by step.", return_tensors="pt").to("cuda")
            outputs = generator_model.generate(**inputs, max_length=200)
            initial_reasoning = generator_tokenizer.decode(outputs[0], skip_special_tokens=True)

            initial_steps = extract_reasoning_steps(initial_reasoning)
            initial_invalid_steps = [
                (i, step) for i, step in enumerate(initial_steps)
                if not predict_validity(question, step, initial_steps[:i] if i > 0 else None, "Logical step.")
            ]

            # Run iterative refinement
            refined_reasoning, refinement_attempts, refinement_time = iterative_refinement(
                generator_model, generator_tokenizer,
                verifier_model, verifier_tokenizer,
                question, predict_validity
            )

            # Verify refined steps
            final_steps = extract_reasoning_steps(refined_reasoning)
            final_invalid_steps = [
                (i, step) for i, step in enumerate(final_steps)
                if not predict_validity(question, step, final_steps[:i] if i > 0 else None, "Logical step.")
            ]

            # Compute metrics
            result = {
                "question": question,
                "initial_reasoning": initial_reasoning,
                "final_reasoning": refined_reasoning,
                "initial_step_count": len(initial_steps),
                "final_step_count": len(final_steps),
                "initial_word_count": len(initial_reasoning.split()),
                "final_word_count": len(refined_reasoning.split()),
                "initial_invalid_count": len(initial_invalid_steps),
                "final_invalid_count": len(final_invalid_steps),
                "reached_valid_reasoning": len(final_invalid_steps) == 0,
                "step_count_change": len(final_steps) - len(initial_steps),
                "word_count_change": len(refined_reasoning.split()) - len(initial_reasoning.split()),
                "initial_readability": textstat.flesch_kincaid_grade(initial_reasoning),
                "final_readability": textstat.flesch_kincaid_grade(refined_reasoning),
                "readability_change": textstat.flesch_kincaid_grade(refined_reasoning) - textstat.flesch_kincaid_grade(initial_reasoning),
                "initial_coherence": coherence_score(initial_steps),
                "final_coherence": coherence_score(final_steps),
                "coherence_change": coherence_score(final_steps) - coherence_score(initial_steps),
                "refinement_attempts": refinement_attempts,
                "refinement_time_seconds": refinement_time
            }
            results.append(result)

        except Exception as e:
            print(f"Error processing example: {e}")

    return pd.DataFrame(results)


In [ ]:
# Run experiment
results_df_reveal = run_experiment(
    llama_model, llama_tokenizer,
    verifier_model, verifier_tokenizer,
    dataset_reveal, num_samples=100,
    predict_validity=predict_validity
)



  0%|          | 0/100 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Iteration 1: Generated Reasoning:
Question: What song did Paul McCartney write for the performer of Valotte?
 Reason step by step.1. Identify the song and performer.2. Determine the relationship between Paul McCartney and the performer.3. Analyze the song's composition and lyrics.
Step 1: Identify the song and performer.
The song is "Valotte" and the performer is 10cc.

Step 2: Determine the relationship between Paul McCartney and the performer.
Paul McCartney is a member of the Beatles and a successful solo artist, but he is not associated with the band 10cc.

Step 3: Analyze the song's composition and lyrics.
"Valotte" is a song written by Graham Lyle and Terry Ritz, and it was recorded by 10cc in 1980.

The final answer is: There is no connection between Paul McCartney and the song "Valotte" for the performer 10cc.
--------------------------------------------------
Extracted Steps for Iteration 1:
Step 1: Step 1: Identify the song and performer. The song is "Valotte" and the perform

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


--------------------------------------------------
Iteration 2: Generated Reasoning:
Q: What song did Paul McCartney write for the performer of Valotte?
- Step 1 is invalid: "Step 1: Identify the song and performer. The song is "Valotte" and the performer is 10cc."
For each invalid step, rewrite it to ensure it follows from the previous steps. Identify incorrect assumptions and provide a revised version. Break down each step logically, verify assumptions, and ensure consistency in your thought process. If any gaps, ambiguities, or contradictions exist, correct them by providing clearer justifications. 
Q: What song did Paul McCartney write for the performer of Valotte?
A: The performer of "Valotte" is 10cc, and Paul McCartney wrote "Valotte" for them. This answer is incorrect because it incorrectly assumes that Paul McCartney wrote the song "Valotte." The correct answer is that Paul McCartney wrote the song "Valotte" for 10cc.

## Step 1: Identify the song and performer.
The
----------

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  1%|          | 1/100 [01:06<1:49:25, 66.32s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


--------------------------------------------------
Iteration 2: Generated Reasoning:
Q: What year was the father of Katherine of England crowned?
- Step 1 is invalid: "Question: What year was the father of Katherine of England crowned?"
- Step 3 is invalid: "The father of Katherine of England was King Henry VIII."
- Step 5 is invalid: "So, the answer is: 1509

Explanation: 
This question requires the reader to follow a series of logical steps to arrive at the correct answer."
- Step 12 is invalid: "In terms of the reasoning skill of Identifying Pros And Cons, this question requires the reader to weigh the"
For each invalid step, rewrite it to ensure it follows from the previous steps. Identify incorrect assumptions and provide a revised version. Break down each step logically, verify assumptions, and ensure consistency in your thought process. If any gaps, ambiguities, or contradictions exist, correct them by providing clearer justifications. 
In this case, the question asks for the ye

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


--------------------------------------------------
Iteration 3: Generated Reasoning:
Q: What year was the father of Katherine of England crowned?
- Step 1 is invalid: "Q: What year was the father of Katherine of England crowned?"
For each invalid step, rewrite it to ensure it follows from the previous steps. Identify incorrect assumptions and provide a revised version. Break down each step logically, verify assumptions, and ensure consistency in your thought process. If any gaps, ambiguities, or contradictions exist, correct them by providing clearer justifications. 
Q: What year was Henry VIII, the father of Katherine of England, crowned? (Assumption: Katherine of England is Henry VIII's daughter)
A: Henry VIII was crowned in 1509.

The final answer is: $\boxed{1509}$
--------------------------------------------------
Extracted Steps for Iteration 3:
Step 1: Q: What year was the father of Katherine of England crowned?
Step 2: - Step 1 is invalid: "Q: What year was the father of Kather

100%|██████████| 100/100 [2:51:42<00:00, 103.03s/it]


In [ ]:
display(results_df_reveal)

,question,initial_reasoning,final_reasoning,initial_step_count,final_step_count,initial_word_count,final_word_count,initial_invalid_count,final_invalid_count,reached_valid_reasoning,step_count_change,word_count_change,initial_readability,final_readability,readability_change,initial_coherence,final_coherence,coherence_change,refinement_attempts,refinement_time_seconds
0,What mass of neutrinos leave the sun every sec...,Q: What mass of neutrinos leave the sun every ...,Q: What mass of neutrinos leave the sun every ...,4,1,146,145,0,0,True,-3,-1,8.8,9.7,0.9,0.465402,1.000000,0.534598,2,56.006903
1,"Is the following sentence plausible? ""Norman P...","Q: Is the following sentence plausible? ""Norma...","Q: Is the following sentence plausible? ""Norma...",5,2,162,154,1,0,True,-3,-8,7.4,9.4,2.0,0.453544,0.762344,0.308800,2,42.976654
2,how fast would you have to be moving to make i...,Q: how fast would you have to be moving to mak...,Question: how fast would you have to be moving...,5,3,129,154,0,0,True,-2,25,6.3,6.5,0.2,0.367861,0.679893,0.312032,1,23.272410
3,How much should I pay back on $100 freely give...,Q: How much should I pay back on $100 freely g...,Question: How much should I pay back on $100 f...,3,3,150,150,0,0,True,0,0,6.4,3.7,-2.7,0.476861,0.551826,0.074965,1,23.594110
4,Could two newborn American Black Bear cubs fit...,Q: Could two newborn American Black Bear cubs ...,Question: Could two newborn American Black Bea...,3,6,147,134,0,0,True,3,-13,7.7,4.6,-3.1,0.510725,0.301008,-0.209717,1,41.308405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,Can an art dealer buy Boeing 737-800 with a Da...,Q: Can an art dealer buy Boeing 737-800 with a...,Question: Can an art dealer buy Boeing 737-800...,3,3,154,158,0,0,True,0,4,10.7,7.6,-3.1,0.557162,0.534999,-0.022164,1,23.591973
92,Who is the chairman of the company that bought...,Q: Who is the chairman of the company that bou...,Q: Who is the chairman of the company that bou...,4,3,113,156,0,0,True,-1,43,7.3,9.0,1.7,0.518026,0.566831,0.048805,2,47.439029
93,Who did the original voice of the character pl...,Q: Who did the original voice of the character...,Question: Who did the original voice of the ch...,4,3,108,150,1,0,True,-1,42,6.9,8.6,1.7,0.542115,0.503231,-0.038885,1,23.494372
94,Does the history of Europe include the age of ...,Q: Does the history of Europe include the age ...,Question: Does the history of Europe include t...,4,2,131,141,0,0,True,-2,10,10.8,12.3,1.5,0.526689,0.689676,0.162987,1,17.814024


In [ ]:
results_df_reveal.to_csv("reveal_dataset_results_new_updated_312.csv", index=False)

